Here is the traditional way to directly call the parent's __init__ method

In [1]:
class MyBaseClass(object):
    
    def __init__(self, value):
        print('MyBaseClass __init__')
        self.value = value
        
class MyChildrenClass(MyBaseClass):
    
    def __init__(self):
        print('MyChildrenClass __init__')
        MyBaseClass.__init__(self, 5)

In [2]:
myChildren = MyChildrenClass()

MyChildrenClass __init__
MyBaseClass __init__


But sometimes this can lead to unpredictable behavior.

###  Issue about the order the parents __init__ are called

In [9]:
class TimesTwo(object):
    
    def __init__(self):
        print('TimesTwo __init__')
        self.value *= 2
        
class PlusFive(object):
    
    def __init__(self):
        print('PlusFive __init__')
        self.value += 5

Class that defines the parent classes in one way

In [10]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [12]:
foo = OneWay(5)
print('First ordering is (5 * 2) + 5 = ', foo.value)

MyBaseClass __init__
TimesTwo __init__
PlusFive __init__
First ordering is (5 * 2) + 5 =  15


Class that defines the parent in another way (call to parents __init__ does not follow class inheritance declaration)

In [13]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [14]:
bar = AnotherWay(5)
print('Second ordering is ', bar.value)

MyBaseClass __init__
TimesTwo __init__
PlusFive __init__
Second ordering is  15


### Issue about diamond inheritance

This happens when a subclass inherits from 2 separate classes that have the same superclass somewhere in the hierarchy. The common superclass's __init__ methods will be ran multiple times ==> causing unexpected behavior.

In [19]:
class TimesFive(MyBaseClass):
    
    def __init__(self, value):
        print('TimesFive __init__')
        MyBaseClass.__init__(self, value)
        self.value *= 5
        
class PlusTwo(MyBaseClass):
    
    def __init__(self, value):
        print('PlusTow __init__')
        MyBaseClass.__init__(self, value)
        self.value += 2

In [21]:
class ThisWay(TimesFive, PlusTwo):
    
    def __init__(self, value):
        print('ThisWay __init__')
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)

In [23]:
foo = ThisWay(5)
print('Should be (5 * 5) + 2 = 27 but it is ', foo.value)

ThisWay __init__
TimesFive __init__
MyBaseClass __init__
PlusTow __init__
MyBaseClass __init__
Should be (5 * 5) + 2 = 27 but it is  7


This is because the second time the superclass __init__ is called, it reset the self.vlaue to 5

### Super (python 2)

MRO: Method Resolution Order

**super** will force to call the super class __init__ only once in a diamond hierarchy

In [30]:
class TimesFiveCorrect(MyBaseClass):
    
    def __init__(self, value):
        print('TimesFiveCorrect __init__')
        super(TimesFiveCorrect, self).__init__(value)
        self.value *= 5
        
class PlusTwoCorrect(MyBaseClass):
    
    def __init__(self, value):
        print('PlusTwoCorrect __init__')
        super(PlusTwoCorrect, self).__init__(value)
        self.value += 2

In [31]:
class GoodWay(TimesFiveCorrect, PlusTwoCorrect):
    
    def __init__(self, value):
        print('GoodWay __init__')
        super(GoodWay, self).__init__(value)

In [32]:
foo = GoodWay(5)
print("Should be 5 * (5+2) = 35 and is ", foo.value)

GoodWay __init__
TimesFiveCorrect __init__
PlusTwoCorrect __init__
MyBaseClass __init__
Should be 5 * (5+2) = 35 and is  35


To check the MRO for this class

In [27]:
from pprint import pprint

This is the order the various classes are called

In [28]:
pprint(GoodWay.mro())

[<class '__main__.GoodWay'>,
 <class '__main__.TimesFiveCorrect'>,
 <class '__main__.PlusTwoCorrect'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]


Then those classes do their work in the opposite order the __init__ were called

### Super (python 3)

In [33]:
class Explicit(MyBaseClass):
    
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)

In [34]:
class Implicit(MyBaseClass):
    
    def __init__(self, value):
        super().__init__(value * 2)